In [1]:
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as mpl

In [8]:
cwd = os.getcwd()
print(cwd)

C:\Users\kayde\OneDrive\Desktop\mannada\RainfallData


In [34]:
variables = [
        'dswr',
        'lftx',
        'mslp',
        'p__f',
        'p__u',
        'p__v',
        'p__z',
        'p_th',
        'p_zh',
        'p5_f',
        'p5_u',
        'p5_v',
        'p5_z',
        'p5th',
        'p5zh',
        'p8_f',
        'p8_u',
        'p8_v',
        'p8_z',
        'p8th',
        'p8zh',
        'p500',
        'p850',
        'pottmp',
        'pr_wtr',
        'prec',
        'r500',
        'r850',
        'rhum',
        'shum',
        'temp',
]
boxes = []
for i in range(3):
    df_li = pd.DataFrame()
    for var in variables:
        path = os.path.join(cwd,"DATA\\BOX_20N_8%.1fE\\ncep_%s.dat"%(2.5*(1+i),var)) # Cleverly curated path for automation
        with open(path) as file:
            temp = file.read().splitlines()
        df_li[var] = temp
    boxes.append(df_li)

In [35]:
boxes[0]

,dswr,lftx,mslp,p__f,p__u,p__v,p__z,p_th,p_zh,p5_f,...,p500,p850,pottmp,pr_wtr,prec,r500,r850,rhum,shum,temp
0,-0.370,1.170,1.466,0.296,-0.821,0.659,-1.120,316.313,-0.570,1.612,...,-1.209,1.090,-1.212,-0.792,0.000,-0.527,0.005,-0.213,-0.259,-1.157
1,-0.719,0.970,1.701,0.780,-1.106,0.767,-1.374,311.375,-0.574,1.657,...,-1.247,1.338,-1.131,-0.636,0.000,-0.829,0.331,-0.185,-0.333,-1.043
2,-1.326,0.726,1.761,0.650,-0.993,0.788,-1.813,314.445,-0.715,1.466,...,-1.058,1.338,-1.334,-0.658,0.000,-0.673,0.755,0.004,-0.265,-1.248
3,-0.672,0.538,1.560,1.041,-0.859,1.274,-1.156,326.051,-1.204,2.369,...,-1.816,1.140,-1.253,-0.852,0.000,-0.751,0.135,-0.034,-0.620,-0.978
4,-0.977,0.206,1.521,0.207,-0.832,0.559,-1.658,313.713,-0.523,2.812,...,-1.664,1.090,-1.273,-0.547,0.838,-0.770,0.723,0.324,-0.503,-1.153
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25563,0.033,2.871,1.397,0.268,-1.198,-0.160,-0.792,285.732,0.044,-0.449,...,1.455,0.966,-1.806,-1.689,0.000,-0.906,-1.562,-0.524,-0.952,-1.855
25564,0.028,2.413,1.406,-0.196,-0.606,0.448,-1.565,318.220,-0.231,-0.625,...,1.805,0.923,-1.506,-1.704,0.000,-0.926,-1.942,-0.647,-1.318,-1.594
25565,-0.178,2.057,1.457,0.390,-0.848,0.717,-1.581,316.781,-0.126,-0.609,...,1.208,0.948,-1.293,-1.630,0.000,-0.916,-1.572,-0.505,-0.732,-1.670
25566,-0.131,2.306,1.324,0.221,-0.942,0.413,-1.205,307.106,0.068,0.376,...,0.175,0.830,-1.070,-1.489,0.000,-1.178,-1.464,-0.892,-1.400,-1.594
